In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

***크롬드라이버 실행(자기 크롬 버전에 맞게 설치후 test.ipynb와 같은 경로에 두기)***

In [ ]:
driver = webdriver.Chrome('chromedriver')
url_list = ["36528615", "31317229", "1269259731", "1779562085", "33040151", "1675801137", "1384361907", "1372982703", "1681990192", "1971062401"]


***클릭할 버튼이 없을때까지 스크롤 내림 (일단 임시로 100지정)***

In [2]:
from selenium.webdriver.common.by import By


In [ ]:
import time
import pandas as pd
userlink = pd.DataFrame()
for _url in url_list[0]:
    URL = f"https://m.place.naver.com/restaurant/{_url}/review/visitor"  
    driver.get(URL)
    time.sleep(2.5)
    for _ in range(100):
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(0.5)
        try: driver.find_element(By.CLASS_NAME, 'fvwqf').click()
        except: pass
        time.sleep(0.1)
    
    link_list = driver.find_elements(By.CLASS_NAME, "Hazns")
    link_list = [link.get_attribute("href") for link in link_list]
    user_list = driver.find_elements(By.CLASS_NAME, "sBWyy")
    user_list = [user.text for user in user_list]
    review_time_list = driver.find_elements(By.CLASS_NAME, "sb8UA")
    review_time_list = [reviews.text for reviews in review_time_list]
    length = min(len(user_list), len(link_list), len(review_time_list))
    userlink2 = pd.DataFrame({'user' : user_list[:length], 'link' : link_list[:length], 'review_time' : review_time_list[:length]}, dtype = str)
    userlink = pd.concat([userlink, userlink2], axis = 0, sort=False)

In [5]:
# iframe 아이디를 반환해줌 
iframes = driver.find_elements(By.CSS_SELECTOR, 'iframe')
for iframe in iframes:
    print(iframe.get_attribute('id'))


searchIframe


In [ ]:
# driver.switch_to.parent_frame() # 아래 코드를 이용해서 iframe갔을때 빠져나오기 위한 코드 (현재는 의미없음)

**식당 리스트 뽑기위한 코드**

In [23]:
import time
import pandas as pd
driver = webdriver.Chrome('chromedriver')
URL = "https://map.naver.com/v5/search/%EA%B0%95%EB%82%A8%EA%B5%AC%20%EC%9D%BC%EC%9B%90%EB%8F%99%20%EC%9D%8C%EC%8B%9D%EC%A0%90?c=14145501.7747606,4507886.3437289,14,0,0,0,dh"  
driver.get(URL)

In [24]:
driver.switch_to.frame('searchIframe') #iframe 으로 이동하기 위한 코드
restaurants_list = pd.DataFrame()

In [25]:
tag_list = driver.find_elements(By.CLASS_NAME, "MVx6e") # 식당 종류 리스트
tag_list = [user.text.split("\n")[-1] for user in tag_list]
tag_list

['4.54리뷰 881TV2TV생생정보',
 '4.36리뷰 999+',
 '곧 영업 시작리뷰 316서울 강남구 일원동',
 '곧 영업 시작리뷰 228서울 강남구 일원동',
 '4.52리뷰 763',
 '영업 전리뷰 744밥이 맛있는 일원동 초밥집',
 '4.77리뷰 319',
 '4.42리뷰 298',
 '4.4리뷰 288',
 '영업 중리뷰 70서울 강남구 일원동']

In [26]:
from tqdm import tqdm
epoch = 6 # 6이 최대인듯
for _ in tqdm(range(epoch)):
    click_list = driver.find_elements(By.CLASS_NAME, "tzwk0")
    click_list[0].click() #아무거나 클릭을 해야 end키가 먹혀서 임의로 아무 클래스 클릭한거임
    for _ in range(10):
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END) # end키를 눌러서 아래까지 로딩
    time.sleep(2)
    click_list = driver.find_elements(By.CLASS_NAME, "tzwk0") # 식당 리스트
    name_list = driver.find_elements(By.CLASS_NAME, "TYaxT") # 식당 이름 리스트
    name_list = [user.text for user in name_list]
    tag_list = driver.find_elements(By.CLASS_NAME, "KCMnt") # 식당 종류 리스트
    tag_list = [user.text for user in tag_list]
    review_list = driver.find_elements(By.CLASS_NAME, "MVx6e") # 리뷰 개수 리스트
    review_list = [user.text.split("\n")[-1] for user in review_list]
    url_list = []
    for restaurant_link in click_list:
        restaurant_link.click()
        url_list.append(driver.current_url)
        time.sleep(0.5)
    restaurants_list2 = pd.DataFrame({'restaurant' : name_list, 'tag' : tag_list, 'url' : url_list, 'review' : review_list}, dtype = str)
    restaurants_list = pd.concat([restaurants_list, restaurants_list2], axis = 0, sort=False)
    driver.find_elements(By.CLASS_NAME, "yUtES")[1].click() #다음페이지 클릭
    time.sleep(2)

100%|██████████| 6/6 [05:22<00:00, 53.75s/it]


In [27]:
restaurants_list.to_csv('river_behind5.csv', index=False)

In [28]:
data = pd.read_csv('./save_csv/river_behind_concat.csv')
url_list = list(data['url'].values)
url_list[:5]

[1675303081, 1867823297, 38969614, 895457986, 11700226]

In [30]:
len(url_list)

3071

In [38]:
userlink = pd.DataFrame()

In [70]:
from tqdm import tqdm
#url_list = ["36528615", "31317229", "1269259731", "1779562085", "33040151", "1675801137", "1384361907", "1372982703", "1681990192", "1971062401"]
current_status = 2000

for _url in tqdm(url_list[2000:2001]):
    if current_status % 50 == 0: #크롬 드라이버 가끔 이상해지는거때문에 50번마다 재실행
        driver = webdriver.Chrome('chromedriver')
        userlink.to_csv('river_behind_concat.csv', index=False)
    URL = f"https://m.place.naver.com/restaurant/{_url}/review/visitor"  
    driver.get(URL)
    time.sleep(2.5)
    prev_height = driver.execute_script("return document. body.scrollHeight")
    while True:
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(0.5)
        try: driver.find_element(By.CLASS_NAME, 'fvwqf').click()
        except: pass
        current_height = driver.execute_script("return document. body.scrollHeight")
        if current_height == prev_height: break
        prev_height = current_height
        time.sleep(0.1)
    
        
    driver.find_elements(By.CLASS_NAME, 'I8cuq')[0].click() #선택 리뷰 창 + 버튼 누르기

    while True:
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(0.5)
        try: driver.find_element(By.CLASS_NAME, 'fvwqf').click()
        except: pass
        current_height = driver.execute_script("return document. body.scrollHeight")
        if current_height == prev_height: break
        prev_height = current_height
        time.sleep(0.1)
        
    time.sleep(0.2)
    link_list = driver.find_elements(By.CLASS_NAME, "Hazns")
    link_list = [link.get_attribute("href") for link in link_list]
    user_list = driver.find_elements(By.CLASS_NAME, "YeINN")
    user_list = [user.text.split("\n") for user in user_list]   
    # while len(user_list) == len(link_list), len(review_time_list), len(review_text_list))
    userlink2 = pd.DataFrame({'link' : link_list, 'user' : user_list}, dtype = str)
    userlink2['rest'] = _url
    userlink = pd.concat([userlink, userlink2], axis = 0, sort=False)
    current_status += 1

100%|██████████| 1/1 [00:39<00:00, 39.51s/it]


In [69]:
userlink

,link,user,rest
0,https://m.place.naver.com/my/5ca8af7af9bd7ae34...,"['돌돔이아빠', '리뷰 158사진 107', '팔로우', '방문자리뷰', '언제나...",1268061360
1,https://m.place.naver.com/my/5e6838228f87a842b...,"['콩이에나', '리뷰 78사진 46', '팔로우', '방문자리뷰', '근처 회사 ...",1268061360
2,https://m.place.naver.com/my/5cc6f1738f87a842b...,"['코코상점 서초점', '리뷰 330사진 55팔로워 3', '팔로우', '방문자리뷰...",1268061360
3,https://m.place.naver.com/my/5ca8af7af9bd7ae34...,"['돌돔이아빠', '리뷰 158사진 107', '팔로우', '뜨아에 반샷 추가 추천...",1268061360
4,https://m.place.naver.com/my/5ca8af7af9bd7ae34...,"['돌돔이아빠', '리뷰 158사진 107', '팔로우', '방문자리뷰', '오늘도...",1268061360
...,...,...,...
201,https://m.place.naver.com/my/5b6876c2a6e162128...,"['딤15', '리뷰 1473사진 22', '팔로우', '별점', '5', '최근 ...",1268061360
202,https://m.place.naver.com/my/5b9cad9f3355424e0...,"['sere25', '리뷰 173사진 1', '팔로우', '별점', '4', '최근...",1268061360
203,https://m.place.naver.com/my/5d25c4a7f66eb146e...,"['호디니', '리뷰 1405사진 3', '팔로우', '별점', '5', '최근 방...",1268061360
204,https://m.place.naver.com/my/5d66829e8f87a842b...,"['나나무라', '리뷰 1464사진 436팔로워 37', '팔로우', '별점', '...",1268061360


In [63]:
driver = webdriver.Chrome('chromedriver')
URL = f"https://m.place.naver.com/restaurant/1641213468/review/visitor"  
driver.get(URL)

In [59]:
user_list = driver.find_elements(By.CLASS_NAME, "YeINN")
user_list = [user.text.split("\n") for user in user_list]
#유저 아이디, 리뷰 000 사진 00 팔로워 00, 필요없는값, 필요없는값, 리뷰내용, 선택리뷰, 필요없는값, 필요없는값, 방문날짜, 방문날짜, 몇번째 방문했는지
user_list[-1]

['Kimyejin',
 '리뷰 204사진 90팔로워 1',
 '팔로우',
 '별점',
 '5',
 '최근 방문일',
 '21.6.7.월',
 '2021년 6월 7일 월요일',
 '1번째 방문영수증']

In [64]:

for click_list in driver.find_elements(By.CLASS_NAME, 'ZGKcF'): #선택 리뷰 창 + 버튼 누르기
    click_list.click()
    time.sleep(0.2)

In [ ]:
for click_list in driver.find_elements(By.CLASS_NAME, 'I8cuq'): #키워드 별점 리뷰 창 더보기 버튼 누르기 (한번만 누르면 됨)
    click_list.click()

In [60]:
user_list[0]

['깔끔한비숑',
 '리뷰 428사진 184팔로워 15',
 '팔로우',
 '방문자리뷰',
 '언제나 맛있는 미칠비루 디저트 카페에요 ! 사장님도 친절하시고 테이블도 새롭게 생겨서 앉아있을 수 있어욤 !! 💕',
 '특별한 메뉴가 있어요디저트가 맛있어요뷰가 좋아요매장이 청결해요친절해요',
 '최근 방문일',
 '22.11.2.수',
 '2022년 11월 2일 수요일',
 '1번째 방문영수증']

In [61]:
user_list[-1]

['Kimyejin',
 '리뷰 204사진 90팔로워 1',
 '팔로우',
 '별점',
 '5',
 '최근 방문일',
 '21.6.7.월',
 '2021년 6월 7일 월요일',
 '1번째 방문영수증']